WebScrapping data using Beautiful Soup, scrap data from html file we downloaded.

we will extract table from that and combine data from aqi data and this data

In [1]:
from ipynb.fs.full.Plot_AQI import avg_data_2013,avg_data_2014,avg_data_2015,avg_data_2016

In [2]:
lst=avg_data_2013()

In [3]:
len(lst)

365

In [10]:
import sys
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import csv

In [17]:
def met_data(month,year):
    file_html=open('Data/Html_Data/{}/{}.html'.format(year,month),'rb') #rb is read byte mode
    plain_text=file_html.read()
    #when using beautiful soup , first we have to open html file and read it
    tempD=[]
    finalD=[]
    #initializing beautifulsoup with plain_text anf 'lxml'
    soup=BeautifulSoup(plain_text,'lxml')
    #we will go to html page and inspect element, seeing the table clause and taking its class
    #here class is- 'medias mensuales numspan'
    for table in soup.findAll('table',{'class':'medias mensuales numspan'}):
        #inside html page we have tbody and tr which have table headings and table rows
        for tbody in table:
            for tr in tbody:  #inside tr we have th(which have specific value) and then we have to pick up text 
                a=tr.get_text()
                tempD.append(a)
                #a will have whole row in it and it have 15 features in it
    rows=len(tempD)/15
    for times in range(round(rows)):
        newtempD=[]
        #inside each row we have 15 features , so we iterate through each feature now
        for i in range(15):
            newtempD.append(tempD[0])
            tempD.pop(0)
        finalD.append(newtempD)
    #we will drop features which have no values in it
    length=len(finalD)
    finalD.pop(length-1) #we not want monthly features in it
    finalD.pop(0) #zeroth record is our feature
    for a in range(len(finalD)):
        finalD[a].pop(14)
        finalD[a].pop(13)
        finalD[a].pop(12)
        finalD[a].pop(11)
        finalD[a].pop(10)
        finalD[a].pop(6)
        finalD[a].pop(0)
    return finalD

In [19]:

def data_combine(year, cs):
    for a in pd.read_csv('Data/Real-Data/real_' + str(year) + '.csv', chunksize=cs):
        df = pd.DataFrame(data=a)
        mylist = df.values.tolist()
    return mylist

In [20]:
if __name__=='__main__':
    if not os.path.exists('Data/real-Data'):
        os.makedirs('Data/real-Data')
    for year in range(2013,2017):
        final_data=[]
        with open('Data/real-Data/real_'+str(year)+'.csv','w') as csvfile:
            wr=csv.writer(csvfile,dialect='excel')
            wr.writerow(
                ['T', 'TM', 'Tm', 'SLP', 'H', 'VV', 'V', 'VM', 'PM 2.5'])
        for month in range(1, 13):
            temp = met_data(month, year)
            final_data = final_data + temp
        #getting dependent features from aqi
        pm = getattr(sys.modules[__name__], 'avg_data_{}'.format(year))()

        if len(pm) == 364:
            pm.insert(364, '-')

        for i in range(len(final_data)-1):
            # final[i].insert(0, i + 1)
            final_data[i].insert(8, pm[i])

        with open('Data/Real-Data/real_' + str(year) + '.csv', 'a') as csvfile:
            wr = csv.writer(csvfile, dialect='excel')
            for row in final_data:
                flag = 0
                for elem in row:
                    if elem == "" or elem == "-":
                        flag = 1
                if flag != 1:
                    wr.writerow(row)
                    
    data_2013 = data_combine(2013, 600)
    data_2014 = data_combine(2014, 600)
    data_2015 = data_combine(2015, 600)
    data_2016 = data_combine(2016, 600)
     
    total=data_2013+data_2014+data_2015+data_2016
    
    with open('Data/Real-Data/Real_Combine.csv', 'w') as csvfile:
        wr = csv.writer(csvfile, dialect='excel')
        wr.writerow(
            ['T', 'TM', 'Tm', 'SLP', 'H', 'VV', 'V', 'VM', 'PM 2.5'])
        wr.writerows(total)
        
        
df=pd.read_csv('Data/Real-Data/Real_Combine.csv')

In [23]:
df=pd.read_csv('Data/real-Data/Real_Combine.csv')
df.head(25)

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833
5,16.0,25.2,10.0,1013.2,79.0,0.6,4.8,11.1,285.225000
6,13.4,21.0,9.2,1015.1,87.0,0.5,1.5,7.6,236.825000
7,14.3,22.6,6.6,1016.3,76.0,0.8,0.4,3.5,276.908333
8,12.7,18.9,7.3,1021.2,76.0,1.8,6.1,16.5,108.000000
9,11.9,18.9,6.3,1020.1,76.0,1.1,8.3,20.6,107.625000


In [24]:
df.shape

(731, 9)